## Spam Detection with RandomForestClassifier and GradientBoostingClassifier

In [3]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [4]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep="\t")
data.columns = ['label','body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    punct_per = round(count/(len(text) - text.count(" ")),3)*100
    return punct_per

data['body_len'] = data['body_text'].apply(lambda x:len(x)-x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x:count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+',text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_features = pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis=1)
X_features.head()


,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Random Forest Classifier and Cross Validation(K-fold)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score

rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf,X_features,data['label'],cv=k_fold,scoring='accuracy',n_jobs=-1)

array([0.97845601, 0.98114901, 0.97574124, 0.96585804, 0.97574124])

### Random Forest Classifier and Holdout Set

In [6]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_features,data['label'],test_size=0.2)

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 50, max_depth = 20, n_jobs = -1)
rf_model = rf.fit(X_train,y_train)

In [9]:
sorted(zip(rf_model.feature_importances_,X_train.columns),reverse=True)[0:10]

[(0.046410778666754915, 'body_len'),
 (0.042657861123862946, 3134),
 (0.035816057499334274, 1803),
 (0.028513935053496583, 7350),
 (0.02142350915664929, 5724),
 (0.02113006782062516, 4796),
 (0.0202156949921851, 7027),
 (0.01946353352389489, 2171),
 (0.0187672808561097, 2031),
 (0.01698232055149229, 5988)]

In [10]:
y_pred = rf_model.predict(X_test)
precision,recall,fscore,support = score(y_test,y_pred,pos_label='spam',average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.582 / Accuracy: 0.941


### Grid Search

In [11]:
def train_RF(n_est,depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth,n_jobs=-1)
    rf_model = rf.fit(X_train,y_train)
    y_pred = rf_model.predict(X_test)
    precision,recall,fscore,support = score(y_test,y_pred,pos_label='spam',average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

for n_est in [10,50,100]:
    for depth in [10,20,30,None]:
        train_RF(n_est,depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.31 / Accuracy: 0.902
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.614 / Accuracy: 0.945
Est: 10 / Depth: 30 ---- Precision: 0.983 / Recall: 0.734 / Accuracy: 0.961
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.797 / Accuracy: 0.971
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.253 / Accuracy: 0.894
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.582 / Accuracy: 0.941
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.696 / Accuracy: 0.957
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.804 / Accuracy: 0.972
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.316 / Accuracy: 0.903
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.614 / Accuracy: 0.945
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.722 / Accuracy: 0.961
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.816 / Accuracy: 0.974


### GridSearchCV

In [14]:
'''from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
param = {'n_estimators':[10,150,300],'max_depth':[30,60,90,None]}

gs = GridSearchCV(rf,param,cv=5,n_jobśśśśśs=-1)
gs_fit = gs.fit(X_features,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]'''

"from sklearn.model_selection import GridSearchCV\nrf = RandomForestClassifier()\nparam = {'n_estimators':[10,150,300],'max_depth':[30,60,90,None]}\n\ngs = GridSearchCV(rf,param,cv=5,n_jobśśśśśs=-1)\ngs_fit = gs.fit(X_features,data['label'])\npd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]"

### Final model selection and evaluation

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125
0,29,3.4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,148,14.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19,10.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31,12.9,0.319345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,45,4.4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score

rf = RandomForestClassifier(n_estimators=150,max_depth=None,n_jobs=-1)
rf_model = rf.fit(X_train_vect,y_train)
y_pred = rf_model.predict(X_test_vect)
precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 1.0 / Recall: 0.801 / Accuracy: 0.973


In [25]:
gb = GradientBoostingClassifier(n_estimators=150,max_depth=11)
gb_model = rf.fit(X_train_vect,y_train)
y_pred = gb_model.predict(X_test_vect)
precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Precision: 1.0 / Recall: 0.801 / Accuracy: 0.973
